<a href="https://colab.research.google.com/github/anvicoder/Data-Science-Assignment-eCommerce-Transactions-/blob/main/Anviksha_Patel__Lookalike_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [72]:
import gdown
customer_id = '1bu_--mo79VdUG9oin4ybfFGRUSXAe-WE'
product_id = '1IKuDizVapw-hyktwfpoAoaGtHtTNHfd0'
transaction_id = '1saEqdbBB-vuk2hxoAf4TzDEsykdKlzbF'
gdown.download(f'https://drive.google.com/uc?id={customer_id}', 'customer.csv', quiet=False)
gdown.download(f'https://drive.google.com/uc?id={product_id}', 'product.csv', quiet=False)
gdown.download(f'https://drive.google.com/uc?id={transaction_id}', 'transaction.csv', quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1bu_--mo79VdUG9oin4ybfFGRUSXAe-WE
To: /content/customer.csv
100%|██████████| 8.54k/8.54k [00:00<00:00, 14.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1IKuDizVapw-hyktwfpoAoaGtHtTNHfd0
To: /content/product.csv
100%|██████████| 4.25k/4.25k [00:00<00:00, 9.53MB/s]
Downloading...
From: https://drive.google.com/uc?id=1saEqdbBB-vuk2hxoAf4TzDEsykdKlzbF
To: /content/transaction.csv
100%|██████████| 54.7k/54.7k [00:00<00:00, 3.97MB/s]


'transaction.csv'

Since, Data already has 0 missing value as found in EDA , there focusing on merging the data and feature engineering.

In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn  as sns
%matplotlib inline

In [74]:
customer_df = pd.read_csv('customer.csv')
product_df = pd.read_csv('product.csv')
transaction_df = pd.read_csv('transaction.csv')
print(customer_df.head())
print(product_df.head())
print(transaction_df.head())

  CustomerID        CustomerName         Region  SignupDate
0      C0001    Lawrence Carroll  South America  2022-07-10
1      C0002      Elizabeth Lutz           Asia  2022-02-13
2      C0003      Michael Rivera  South America  2024-03-07
3      C0004  Kathleen Rodriguez  South America  2022-10-09
4      C0005         Laura Weber           Asia  2022-08-15
  ProductID              ProductName     Category   Price
0      P001     ActiveWear Biography        Books  169.30
1      P002    ActiveWear Smartwatch  Electronics  346.30
2      P003  ComfortLiving Biography        Books   44.12
3      P004            BookWorld Rug   Home Decor   95.69
4      P005          TechPro T-Shirt     Clothing  429.31
  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3       

In [75]:
merged_df1 = pd.merge(transaction_df, customer_df, on='CustomerID', how='left')
final_df = pd.merge(merged_df1, product_df, on='ProductID', how='left')
print(final_df.head())
print(final_df.info())
final_df.dtypes

  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue  Price_x     CustomerName         Region  SignupDate  \
0      300.68   300.68   Andrea Jenkins         Europe  2022-12-03   
1      300.68   300.68  Brittany Harvey           Asia  2024-09-04   
2      300.68   300.68  Kathryn Stevens         Europe  2024-04-04   
3      601.36   300.68  Travis Campbell  South America  2024-04-11   
4      902.04   300.68    Timothy Perez         Europe  2022-03-15   

                       ProductName     Category  Price_y  
0  ComfortLiving Bluetooth Speaker  Electronics   300.68  
1  ComfortLiving Bluetooth Speaker

,0
TransactionID,object
CustomerID,object
ProductID,object
TransactionDate,object
Quantity,int64
TotalValue,float64
Price_x,float64
CustomerName,object
Region,object
SignupDate,object


In [76]:
print(final_df.columns)


Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       'Quantity', 'TotalValue', 'Price_x', 'CustomerName', 'Region',
       'SignupDate', 'ProductName', 'Category', 'Price_y'],
      dtype='object')


In [77]:
transaction_agg = final_df.groupby('CustomerID').agg(
    transaction_frequency=('TransactionID', 'nunique'),
    total_spend=('TotalValue', 'sum')
).reset_index()

In [64]:
product_categories = final_df.groupby('CustomerID')['Category'].apply(lambda x: ', '.join(x.unique())).reset_index()

In [78]:
final_df = pd.merge(final_df[['CustomerID', 'Region']], transaction_agg, on='CustomerID', how='left')
final_df = pd.merge(final_df, product_categories, on='CustomerID', how='left')
print(final_df[['CustomerID', 'Region', 'transaction_frequency', 'total_spend', 'Category']].head())
# Feature Scaling
scaler = MinMaxScaler()
final_df[['transaction_frequency_scaled', 'total_spend_scaled']] = scaler.fit_transform(final_df[['transaction_frequency', 'total_spend']])
print(final_df[['CustomerID', 'transaction_frequency_scaled', 'total_spend_scaled']].head())


  CustomerID         Region  transaction_frequency  total_spend  \
0      C0199         Europe                      4      1979.28   
1      C0146           Asia                      4      2570.80   
2      C0127         Europe                      6      3232.88   
3      C0087  South America                      7      6604.23   
4      C0070         Europe                      4      3125.49   

                         Category  
0         Electronics, Home Decor  
1  Electronics, Home Decor, Books  
2  Electronics, Home Decor, Books  
3    Electronics, Books, Clothing  
4              Electronics, Books  
  CustomerID  transaction_frequency_scaled  total_spend_scaled
0      C0199                           0.3            0.179098
1      C0146                           0.3            0.234947
2      C0127                           0.5            0.297457
3      C0087                           0.6            0.615764
4      C0070                           0.3            0.287318


In [68]:
print(final_df[['CustomerID', 'Region', 'transaction_frequency', 'total_spend', 'Category']].head())


  CustomerID         Region  transaction_frequency  total_spend  \
0      C0199         Europe                      4      1979.28   
1      C0146           Asia                      4      2570.80   
2      C0127         Europe                      6      3232.88   
3      C0087  South America                      7      6604.23   
4      C0070         Europe                      4      3125.49   

                         Category  
0         Electronics, Home Decor  
1  Electronics, Home Decor, Books  
2  Electronics, Home Decor, Books  
3    Electronics, Books, Clothing  
4              Electronics, Books  


In [79]:
scaler = MinMaxScaler()
final_df[['transaction_frequency_scaled', 'total_spend_scaled']] = scaler.fit_transform(final_df[['transaction_frequency', 'total_spend']])
print(final_df[['CustomerID', 'transaction_frequency_scaled', 'total_spend_scaled']].head())

  CustomerID  transaction_frequency_scaled  total_spend_scaled
0      C0199                           0.3            0.179098
1      C0146                           0.3            0.234947
2      C0127                           0.5            0.297457
3      C0087                           0.6            0.615764
4      C0070                           0.3            0.287318


In [71]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
# Step 1: Feature Scaling
scaler = MinMaxScaler()
final_df[['transaction_frequency_scaled', 'total_spend_scaled']] = scaler.fit_transform(final_df[['transaction_frequency', 'total_spend']])

# Step 2: KNN for Similarity Calculation
features = final_df[['transaction_frequency_scaled', 'total_spend_scaled']]
knn = NearestNeighbors(n_neighbors=4, metric='euclidean')
knn.fit(features)
distances, indices = knn.kneighbors(features)

# Step 3: Filter for the first 20 customers
lookalike_map = []
epsilon = 1e-6

for i in range(20):
    customer_id = final_df['CustomerID'].iloc[i]
    similar_customers = final_df['CustomerID'].iloc[indices[i][1:]]
    adjusted_distances = distances[i][1:] + epsilon
    similarity_scores = 1 / adjusted_distances
    for similar_customer, score in zip(similar_customers, similarity_scores):
        lookalike_map.append([customer_id, similar_customer, score])
lookalike_df = pd.DataFrame(lookalike_map, columns=['cust_id', 'similar_customer_id', 'similarity_score'])
print(lookalike_df.head())
lookalike_df.to_csv('Lookalike.csv', index=False)

  cust_id similar_customer_id  similarity_score
0   C0199               C0199         1000000.0
1   C0199               C0199         1000000.0
2   C0199               C0199         1000000.0
3   C0146               C0146         1000000.0
4   C0146               C0146         1000000.0
